In [1]:
import pandas as pd

In [2]:
pd.__version__

'1.4.2'

In [5]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet')

In [4]:
# if error run:
#!pip install pyarrow # or fastparquet

     |████████████████████████████████| 40.8 MB 24.4 MB/s eta 0:00:01
